ReadMe file: https://www.seanlahman.com/files/database/readme2021.txt

In [21]:
# pip install -r requirements.txt

     |████████████████████████████████| 11.3 MB 22.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 16.8 MB 84.8 MB/s            
  Created wheel for duckdb: filename=duckdb-0.3.3-cp39-cp39-linux_x86_64.whl size=12926964 sha256=b6ce62deac32065e31df19958f10d28f7ddbf222a84191ffe11f09ee6359d186
  Stored in directory: /home/jovyan/.cache/pip/wheels/37/47/74/8419272484edff0f026281885b65610b5c28803492fa7d30d0
Successfully built duckdb
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.5.2.dev1312
    Uninstalling duckdb-0.5.2.dev1312:
      Successfully uninstalled duckdb-0.5.2.dev1312
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geopandas 0.11

In [ ]:
#import shutil
#dir_name = "/home/jovyan/work/BigData/lahnman-to-duckdb"
#output_filename = 'zipped_work'
#shutil.make_archive(output_filename, 'zip', dir_name)

In [ ]:
import csv
from importlib.resources import files
from itertools import chain
from pathlib import PosixPath

import duckdb
import os

In [ ]:
os.getcwd()
%cd "/home/jovyan/work/BigData/lahnman-to-duckdb"

In [ ]:
SQL = os.listdir('src/sql')
CORE = os.listdir('src/lahnman/core/')
CONTRIB = os.listdir('src/lahnman/contrib/')
TMP = os.listdir('src/tmp/')

In [ ]:
TABLE_ORDER = ['src/lahnman/core/people', 'src/lahnman/core/parks', 'src/lahnman/core/teamsfranchises', 'src/lahnman/core/teams',
               'src/lahnman/contrib/salaries', 'src/lahnman/contrib/schools', 'src/lahnman/core/allstarfull', 'src/lahnman/core/appearances',
               'src/lahnman/contrib/awardsmanagers', 'src/lahnman/contrib/awardsplayers', 'src/lahnman/contrib/awardssharemanagers', 
               'src/lahnman/contrib/awardsshareplayers', 'src/lahnman/core/batting', 'src/lahnman/core/battingpost',
               'src/lahnman/contrib/collegeplaying', 'src/lahnman/core/fielding', 'src/lahnman/core/fieldingof', 
               'src/lahnman/core/fieldingofsplit', 'src/lahnman/core/fieldingpost', 'src/lahnman/contrib/halloffame',
               'src/lahnman/core/homegames','src/lahnman/core/managers', 'src/lahnman/core/managershalf', 'src/lahnman/core/pitching',
               'src/lahnman/core/pitchingpost', 'src/lahnman/core/seriespost', 'src/lahnman/core/teamshalf']

In [ ]:
def open_query(fname: PosixPath) -> str:
    with open(fname) as f:
        query = f.read()
    return query


def parse_query_columns(query_path: PosixPath) -> list[str]:
    """Return a list of columns for a given create table statement"""
    cols = []
    query = open_query(query_path)
    if not query.strip().startswith('CREATE TABLE'):
        return cols

    for row in query.split('\n'):
        if row.strip().startswith('"'):
            cols.append(row.strip().split('"')[1])
    return cols


def create_table_data_mapping() -> dict:
    """Return mapping of all table names -> query_path, file_path, create statement columns"""
    queries = ['src/sql/' + each for each in SQL]

    core = ['src/lahnman/core/' +  each for each in CORE]
    contrib = ['src/lahnman/contrib/' + each for each in CONTRIB]
    core.extend(contrib)
    files = core.copy()

    d = {}
    for fname, query in zip(files, queries):  # excludes final index query bc there is no associated file path
        name = fname.replace('.csv', '').lower()

        d[name] = {
            'query_path': query,
            'file_path': fname,
            'cols': parse_query_columns(query),
        }

    return d


def load_all_create_statements(mapping: dict):
    """Join all create statements into a single string"""
    all_queries = ''
    for table in TABLE_ORDER:
        all_queries += open_query(mapping.get(table)['query_path']) + '\n'
    return all_queries


def increment_and_save_csv(fname: PosixPath, fout: PosixPath) -> None:
    """Load csv, add zero-index "ID" column, increment starting at 1, return header and incremented data tuple"""
    results = []

    with open(fout, 'wt') as fo:
        csvwriter = csv.writer(fo)

        with open(fname) as f:
            csvreader = csv.reader(f)
            header = ['ID'] + next(csvreader)
            csvwriter.writerow(header)

            for i, row in enumerate(csvreader, start=1):
                csvwriter.writerow([i] + row)


def main():
    conn = duckdb.connect(database='lahnman.duckdb')
    mapping = create_table_data_mapping()

    create_statements = load_all_create_statements(mapping)
    
    create_statements = create_statements.replace('\n', '')
    create_statements = create_statements.replace('"', '')
    create_statements = ' '.join(create_statements.split())
    
    conn.execute('BEGIN TRANSACTION')
    conn.execute(create_statements)
    conn.execute('COMMIT')
    
    conn.execute('BEGIN TRANSACTION')
    for table in TABLE_ORDER:
        d = mapping.get(table)
        print(table)

        file_path = d['file_path']
        tmp_path = TMP.joinpath(file_path.name)

        # re-write file in tmp/ with ID col and incremented value
        increment_and_save_csv(file_path, tmp_path)

        cols = [c for c in d['cols'] if '_ID' not in c]  # trim foreign keys but leave primary ID

        conn.execute(f"COPY {table}({','.join(cols)}) FROM '{str(tmp_path)}' (DELIMITER ',', HEADER 1);")
    conn.execute('COMMIT')

    # final index query
    conn.execute(open_query(SQL.joinpath('zzz_indicies.sql')))

In [ ]:
conn = duckdb.connect(database='lahnman.duckdb')
mapping = create_table_data_mapping()
conn.execute('BEGIN TRANSACTION')
conn.execute(abbrev_create)
conn.execute('COMMIT')

In [250]:
abbrev_create = 'CREATE TABLE people (playerID INTEGER NOT NULL, birthYear INTEGER NOT NULL); CREATE TABLE teamsfranchises ( ID INTEGER NOT NULL, franchID VARCHAR(3) NOT NULL, franchName VARCHAR(50) NULL, active CHARACTER(1) NULL, NAassoc VARCHAR(3) NULL, PRIMARY KEY (ID));CREATE TABLE teams ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NULL, teamID CHARACTER(3) NOT NULL, franchID VARCHAR(3) NULL, divID CHARACTER(1) NULL, div_ID INTEGER NULL, teamRank SMALLINT NULL, G SMALLINT NULL, Ghome SMALLINT NULL, W SMALLINT NULL, L SMALLINT NULL, DivWin VARCHAR(1) NULL, WCWin VARCHAR(1) NULL, LgWin VARCHAR(1) NULL, WSWin VARCHAR(1) NULL, R SMALLINT NULL, AB SMALLINT NULL, H SMALLINT NULL, B2 SMALLINT NULL, B3 SMALLINT NULL, HR SMALLINT NULL, BB SMALLINT NULL, SO SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, HBP SMALLINT NULL, SF SMALLINT NULL, RA SMALLINT NULL, ER SMALLINT NULL, ERA DOUBLE NULL, CG SMALLINT NULL, SHO SMALLINT NULL, SV SMALLINT NULL, IPouts INTEGER NULL, HA SMALLINT NULL, HRA SMALLINT NULL, BBA SMALLINT NULL, SOA SMALLINT NULL, E INTEGER NULL, DP INTEGER NULL, FP DOUBLE NULL, name VARCHAR(50) NULL, park VARCHAR(255) NULL, attendance INTEGER NULL, BPF INTEGER NULL, PPF INTEGER NULL, teamIDBR VARCHAR(3) NULL, teamIDlahman45 VARCHAR(3) NULL, teamIDretro VARCHAR(3) NULL, PRIMARY KEY (ID), FOREIGN KEY(franchID) REFERENCES teamsfranchises (franchID) ON UPDATE NO ACTION ON DELETE NO ACTION);'

In [246]:
create_statements = 'CREATE TABLE people (playerID INTEGER NOT NULL, birthYear INTEGER NOT NULL); CREATE TABLE teamsfranchises ( ID INTEGER NOT NULL, franchID VARCHAR(3) NOT NULL, franchName VARCHAR(50) NULL, active CHARACTER(1) NULL, NAassoc VARCHAR(3) NULL, PRIMARY KEY (ID));CREATE TABLE teams ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NULL, teamID CHARACTER(3) NOT NULL, franchID VARCHAR(3) NULL, divID CHARACTER(1) NULL, div_ID INTEGER NULL, teamRank SMALLINT NULL, G SMALLINT NULL, Ghome SMALLINT NULL, W SMALLINT NULL, L SMALLINT NULL, DivWin VARCHAR(1) NULL, WCWin VARCHAR(1) NULL, LgWin VARCHAR(1) NULL, WSWin VARCHAR(1) NULL, R SMALLINT NULL, AB SMALLINT NULL, H SMALLINT NULL, B2 SMALLINT NULL, B3 SMALLINT NULL, HR SMALLINT NULL, BB SMALLINT NULL, SO SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, HBP SMALLINT NULL, SF SMALLINT NULL, RA SMALLINT NULL, ER SMALLINT NULL, ERA DOUBLE NULL, CG SMALLINT NULL, SHO SMALLINT NULL, SV SMALLINT NULL, IPouts INTEGER NULL, HA SMALLINT NULL, HRA SMALLINT NULL, BBA SMALLINT NULL, SOA SMALLINT NULL, E INTEGER NULL, DP INTEGER NULL, FP DOUBLE NULL, name VARCHAR(50) NULL, park VARCHAR(255) NULL, attendance INTEGER NULL, BPF INTEGER NULL, PPF INTEGER NULL, teamIDBR VARCHAR(3) NULL, teamIDlahman45 VARCHAR(3) NULL, teamIDretro VARCHAR(3) NULL, PRIMARY KEY (ID), FOREIGN KEY(franchID) REFERENCES teamsfranchises (franchID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE schools ( ID INTEGER NOT NULL, schoolID VARCHAR(15) NOT NULL, name_full VARCHAR(255) NULL, city VARCHAR(55) NULL, state VARCHAR(55) NULL, country VARCHAR(55) NULL, PRIMARY KEY (ID));CREATE TABLE parks ( ID INTEGER NOT NULL, parkalias VARCHAR(255) NULL, parkkey VARCHAR(255) NULL, parkname VARCHAR(255) NULL, city VARCHAR(255) NULL, state VARCHAR(255) NULL, country VARCHAR(255) NULL, PRIMARY KEY (ID));CREATE TABLE homegames ( ID INTEGER NOT NULL, yearkey INTEGER NULL, leaguekey CHARACTER(2) NULL, teamkey CHARACTER(3) NULL, team_ID INTEGER NULL, parkkey VARCHAR(255) NULL, park_ID INTEGER NULL, spanfirst VARCHAR(255) NULL, spanlast VARCHAR(255) NULL, games INTEGER NULL, openings INTEGER NULL, attendance INTEGER NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(park_ID) REFERENCES parks (ID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE awardssharemanagers ( ID INTEGER NOT NULL, awardID VARCHAR(25) NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NOT NULL, playerID VARCHAR(10) NOT NULL, pointsWon SMALLINT NULL, pointsMax SMALLINT NULL, votesFirst SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE fieldingof ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, stint SMALLINT NOT NULL, Glf SMALLINT NULL, Gcf SMALLINT NULL, Grf SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE fieldingofsplit ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, stint SMALLINT NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, POS VARCHAR(2) NOT NULL, G SMALLINT NULL, GS SMALLINT NULL, InnOuts SMALLINT NULL, PO SMALLINT NULL, A SMALLINT NULL, E SMALLINT NULL, DP SMALLINT NULL, PB SMALLINT NULL, WP SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, ZR DOUBLE NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE awardsplayers ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, awardID VARCHAR(255) NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NULL, tie VARCHAR(1) NULL, notes VARCHAR(100) NULL, PRIMARY KEY (ID), FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE pitchingpost ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, round VARCHAR(10) NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, W SMALLINT NULL, L SMALLINT NULL, G SMALLINT NULL, GS SMALLINT NULL, CG SMALLINT NULL, SHO SMALLINT NULL, SV SMALLINT NULL, IPouts INTEGER NULL, H SMALLINT NULL, ER SMALLINT NULL, HR SMALLINT NULL, BB SMALLINT NULL, SO SMALLINT NULL, BAOpp DOUBLE NULL, ERA DOUBLE NULL, IBB SMALLINT NULL, WP SMALLINT NULL, HBP SMALLINT NULL, BK SMALLINT NULL, BFP SMALLINT NULL, GF SMALLINT NULL, R SMALLINT NULL, SH SMALLINT NULL, SF SMALLINT NULL, GIDP SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE managershalf ( ID INTEGER NOT NULL, playerID VARCHAR(10) NOT NULL, yearID SMALLINT NOT NULL, teamID CHARACTER(3) NOT NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, inseason SMALLINT NULL, half SMALLINT NOT NULL, G SMALLINT NULL, W SMALLINT NULL, L SMALLINT NULL, teamRank SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE teamshalf ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NOT NULL, teamID CHARACTER(3) NOT NULL, team_ID INTEGER NULL, Half VARCHAR(1) NOT NULL, divID CHARACTER(1) NULL, div_ID INTEGER NULL, DivWin VARCHAR(1) NULL, teamRank SMALLINT NULL, G SMALLINT NULL, W SMALLINT NULL, L SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE appearances ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, teamID CHARACTER(3) NOT NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, playerID VARCHAR(9) NOT NULL, G_all SMALLINT NULL, GS SMALLINT NULL, G_batting SMALLINT NULL, G_defense SMALLINT NULL, G_p SMALLINT NULL, G_c SMALLINT NULL, G_1b SMALLINT NULL, G_2b SMALLINT NULL, G_3b SMALLINT NULL, G_ss SMALLINT NULL, G_lf SMALLINT NULL, G_cf SMALLINT NULL, G_rf SMALLINT NULL, G_of SMALLINT NULL, G_dh SMALLINT NULL, G_ph SMALLINT NULL, G_pr SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE salaries ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, teamID CHARACTER(3) NOT NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NOT NULL, playerID VARCHAR(9) NOT NULL, salary DOUBLE NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE fieldingpost ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, round VARCHAR(10) NOT NULL, POS VARCHAR(2) NOT NULL, G SMALLINT NULL, GS SMALLINT NULL, InnOuts SMALLINT NULL, PO SMALLINT NULL, A SMALLINT NULL, E SMALLINT NULL, DP SMALLINT NULL, TP SMALLINT NULL, PB SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE pitching ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, stint SMALLINT NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, W SMALLINT NULL, L SMALLINT NULL, G SMALLINT NULL, GS SMALLINT NULL, CG SMALLINT NULL, SHO SMALLINT NULL, SV SMALLINT NULL, IPouts INTEGER NULL, H SMALLINT NULL, ER SMALLINT NULL, HR SMALLINT NULL, BB SMALLINT NULL, SO SMALLINT NULL, BAOpp DOUBLE NULL, ERA DOUBLE NULL, IBB SMALLINT NULL, WP SMALLINT NULL, HBP SMALLINT NULL, BK SMALLINT NULL, BFP SMALLINT NULL, GF SMALLINT NULL, R SMALLINT NULL, SH SMALLINT NULL, SF SMALLINT NULL, GIDP SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE seriespost ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, round VARCHAR(5) NOT NULL, teamIDwinner VARCHAR(3) NULL, lgIDwinner VARCHAR(2) NULL, team_IDwinner INTEGER NULL, teamIDloser VARCHAR(3) NULL, team_IDloser INTEGER NULL, lgIDloser VARCHAR(2) NULL, wins SMALLINT NULL, losses SMALLINT NULL, ties SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_IDwinner) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(team_IDloser) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE awardsshareplayers ( ID INTEGER NOT NULL, awardID VARCHAR(25) NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NOT NULL, playerID VARCHAR(9) NOT NULL, pointsWon DOUBLE NULL, pointsMax SMALLINT NULL, votesFirst DOUBLE NULL, PRIMARY KEY (ID), FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE managers ( ID INTEGER NOT NULL, playerID VARCHAR(10) NULL, yearID SMALLINT NOT NULL, teamID CHARACTER(3) NOT NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, inseason SMALLINT NOT NULL, G SMALLINT NULL, W SMALLINT NULL, L SMALLINT NULL, teamRank SMALLINT NULL, plyrMgr VARCHAR(1) NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE collegeplaying ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, schoolID VARCHAR(15) NULL, yearID SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(schoolID) REFERENCES schools (schoolID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE fielding ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, stint SMALLINT NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, POS VARCHAR(2) NOT NULL, G SMALLINT NULL, GS SMALLINT NULL, InnOuts SMALLINT NULL, PO SMALLINT NULL, A SMALLINT NULL, E SMALLINT NULL, DP SMALLINT NULL, PB SMALLINT NULL, WP SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, ZR DOUBLE NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE battingpost ( ID INTEGER NOT NULL, yearID SMALLINT NOT NULL, round VARCHAR(10) NOT NULL, playerID VARCHAR(9) NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, G SMALLINT NULL, AB SMALLINT NULL, R SMALLINT NULL, H SMALLINT NULL, B2 SMALLINT NULL, B3 SMALLINT NULL, HR SMALLINT NULL, RBI SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, BB SMALLINT NULL, SO SMALLINT NULL, IBB SMALLINT NULL, HBP SMALLINT NULL, SH SMALLINT NULL, SF SMALLINT NULL, GIDP SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE awardsmanagers ( ID INTEGER NOT NULL, playerID VARCHAR(10) NOT NULL, awardID VARCHAR(75) NOT NULL, yearID SMALLINT NOT NULL, lgID CHARACTER(2) NOT NULL, tie VARCHAR(1) NULL, notes VARCHAR(100) NULL, PRIMARY KEY (ID), FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE INDEX allstarfull_team_ID ON allstarfull (team_ID);CREATE INDEX appearances_playerID ON appearances (playerID);CREATE INDEX appearances_team_ID ON appearances (team_ID);CREATE UNIQUE INDEX appearances_yearID ON appearances (yearID, teamID, playerID);CREATE UNIQUE INDEX batting_playerID ON batting (playerID, yearID, stint);CREATE INDEX batting_team_ID ON batting (team_ID);CREATE INDEX battingpost_playerID ON battingpost (playerID);CREATE INDEX battingpost_team_ID ON battingpost (team_ID);CREATE UNIQUE INDEX battingpost_yearID ON battingpost (yearID, round, playerID);CREATE INDEX collegeplaying_playerID ON collegeplaying (playerID);CREATE INDEX collegeplaying_schoolID ON collegeplaying (schoolID);CREATE UNIQUE INDEX fielding_playerID ON fielding (playerID, yearID, stint, POS);CREATE INDEX fielding_team_ID ON fielding (team_ID);CREATE UNIQUE INDEX fieldingof_playerID ON fieldingof (playerID, yearID, stint);CREATE UNIQUE INDEX fieldingofsplit_playerID ON fieldingofsplit (playerID, yearID, stint, POS);CREATE INDEX fieldingofsplit_team_ID ON fieldingofsplit (team_ID);CREATE UNIQUE INDEX fieldingpost_playerID ON fieldingpost (playerID, yearID, round, POS);CREATE INDEX fieldingpost_team_ID ON fieldingpost (team_ID);CREATE UNIQUE INDEX halloffame_playerID ON halloffame (playerID, yearid, votedBy);CREATE INDEX homegames_park_ID ON homegames (park_ID);CREATE INDEX homegames_team_ID ON homegames (team_ID);CREATE INDEX managers_playerID ON managers (playerID);CREATE INDEX managers_team_ID ON managers (team_ID);CREATE UNIQUE INDEX managers_yearID ON managers (yearID, teamID, inseason);CREATE UNIQUE INDEX managershalf_playerID ON managershalf (playerID, yearID, teamID, half);CREATE INDEX managershalf_team_ID ON managershalf (team_ID);CREATE UNIQUE INDEX pitching_playerID ON pitching (playerID, yearID, stint);CREATE INDEX pitching_team_ID ON pitching (team_ID);CREATE UNIQUE INDEX pitchingpost_playerID ON pitchingpost (playerID, yearID, round);CREATE INDEX pitchingpost_team_ID ON pitchingpost (team_ID);CREATE INDEX salaries_playerID ON salaries (playerID);CREATE INDEX salaries_team_ID ON salaries (team_ID);CREATE INDEX seriespost_team_IDloser ON seriespost (team_IDloser);CREATE INDEX seriespost_team_IDwinner ON seriespost (team_IDwinner);CREATE UNIQUE INDEX seriespost_yearID ON seriespost (yearID, round);CREATE INDEX teams_div_ID ON teams (div_ID);CREATE INDEX teams_franchID ON teams (franchID);CREATE INDEX teamshalf_div_ID ON teamshalf (div_ID);CREATE INDEX teamshalf_team_ID ON teamshalf (team_ID);CREATE TABLE batting ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NOT NULL, stint SMALLINT NOT NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, G SMALLINT NULL, AB SMALLINT NULL, R SMALLINT NULL, H SMALLINT NULL, B2 SMALLINT NULL, B3 SMALLINT NULL, HR SMALLINT NULL, RBI SMALLINT NULL, SB SMALLINT NULL, CS SMALLINT NULL, BB SMALLINT NULL, SO SMALLINT NULL, IBB SMALLINT NULL, HBP SMALLINT NULL, SH SMALLINT NULL, SF SMALLINT NULL, GIDP SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION, FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE allstarfull ( ID INTEGER NOT NULL, playerID VARCHAR(9) NOT NULL, yearID SMALLINT NULL, gameNum SMALLINT NOT NULL, gameID VARCHAR(12) NULL, teamID CHARACTER(3) NULL, team_ID INTEGER NULL, lgID CHARACTER(2) NULL, GP SMALLINT NULL, startingPos SMALLINT NULL, PRIMARY KEY (ID), FOREIGN KEY(team_ID) REFERENCES teams (ID) ON UPDATE NO ACTION ON DELETE NO ACTION);CREATE TABLE halloffame ( ID INTEGER NOT NULL, playerID VARCHAR(10) NOT NULL, yearid SMALLINT NOT NULL, votedBy VARCHAR(64) NOT NULL, ballots SMALLINT NULL, needed SMALLINT NULL, votes SMALLINT NULL, inducted VARCHAR(1) NULL, category VARCHAR(20) NULL, needed_note VARCHAR(25) NULL, PRIMARY KEY (ID), FOREIGN KEY(playerID) REFERENCES people (playerID) ON UPDATE NO ACTION ON DELETE NO ACTION);'

In [ ]:
if __name__ == '__main__':
    main()